In [ ]:
spark

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test = pd.read_csv("X/Esg_ANAll!Disclosure_Score_M10.csv", index_col=0)
test = test.fillna(0)
test



In [ ]:
c = test.columns.values
c


In [ ]:
c.size
d.size

In [ ]:
e = np.unique(d)
e.size

In [ ]:
g = np.append(d,'IZ389654')

In [ ]:
h = np.unique(g)
h.size

In [ ]:
h = np.delete(h, 0)
np.sort(h)

In [ ]:
import glob
import numpy as np
import pandas as pd

########## SCAN AND CONSTRUCT MATRIX FORM ######################
filesList = glob.glob("X/*.csv")

ix = []
col = []
indexList = np.array(ix)
colList = np.array(col)

for f_ in filesList:
    print(f_)
    df = pd.read_csv(f_, index_col=0)
    indexList = np.append(indexList, df.index.values)
    indexList = np.unique(indexList)
    indexList = np.sort(indexList)    
    
    colList = np.append(colList, df.columns.values)
    colList = np.unique(colList)
    colList = np.sort(colList)
    
##### Rustine pour drop CIQID
colList = np.delete(colList,0)
##############################

## Generate the DF for double index stacking
df = pd.DataFrame(index=indexList,columns=colList, data=0)
temp = df.stack()

## To drop the unwanted 0 column
temp.to_csv("X_output/_model.csv")
df = pd.read_csv("X_output/_model.csv")

T = df.drop("0", axis=1)

#FINAL MODEL
T.to_csv("X_output/model.csv", index=False)   
os.remove("X_output/_model.csv")
    

In [ ]:
def fill_in(model,df_wide):
    index = df_wide.index.values
    
    df = model
    df['value'] = 0
    
    for i in range(len(index)):
        a = df_wide.index.values[i]
        print(a)
        print(i)
        df.ix[a].value = df_wide.ix[a].values
        
    return df

In [ ]:
import numpy as np
import pandas as pd
import glob


filesList = glob.glob("X/*.csv")

total = len(filesList)
i = 0
for f_ in filesList:
    i = i + 1
    print("Work: " + str(i) + "/" + str(total))
    print(f_)
    f_c = f_.replace('.csv','')
    f_c = f_c.replace('X/','')
    f_c_ = ''.join(e for e in f_c if e.isalnum())


    test = pd.read_csv(f_, index_col=0)
    if 'CIQID' in test.columns:
        del test['CIQID']
    
    test = test.fillna(0)
    test = test.stack().to_frame()

    model = pd.read_csv("X_output/model.csv", index_col=[0,1])

    df = pd.DataFrame(index=model.index.difference(test.index),data=0,columns=[0])

    frames = [test, df]
    bDF = pd.concat(frames)
    bDF.sort_index()
    bDF.columns = [f_c_]
    bDF.to_csv("X_output/standalone/"+f_c+".csv",index=True)
    bDF.to_csv("X_output/workfile/"+f_c+".csv",index=False)  

In [ ]:
model.index.values

In [ ]:
type(model.index.difference(test.index))

In [ ]:
len(model.index.values)

In [ ]:
len(test.index.values)

In [ ]:
sc

In [ ]:
spark

In [ ]:
df1 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/workA.csv")
df1.show()

df2 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/workB.csv")
df2.show()

df3 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/standA.csv")
df3.show()

df4 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/standB.csv")
df4.show()

df3 = df3.withColumnRenamed('_c0','date')
df3 = df3.withColumnRenamed('_c1','Action')
df4 = df4.withColumnRenamed('_c0','ix0')
df4 = df4.withColumnRenamed('_c1','ix1')

In [ ]:
df3.createOrReplaceTempView("d3")
df4.createOrReplaceTempView("d4")

keep = df3.columns[0] + "," + df3.columns[1] + "," + df3.columns[2] + "," + df4.columns[2]

query = "SELECT {} FROM d3 d LEFT JOIN d4 e ON d.date = e.ix0 AND d.Action = e.ix1".format(keep)
print(query)

sqlContext.sql(query).show()

In [ ]:
df3 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/standA.csv")
df3.show()

df4 = spark.read.format("csv")\
        .option("header", "true")\
        .option("mode", "PERMISSIVE")\
        .option("inferSchema", "true")\
        .option("delimiter", ",")\
        .csv("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/standB.csv")
df4.show()

df3 = df3.withColumnRenamed('_c0','date')
df3 = df3.withColumnRenamed('_c1','Action')
df4 = df4.withColumnRenamed('_c0','ix0')
df4 = df4.withColumnRenamed('_c1','ix1')

df3.createOrReplaceTempView("d3")
df4.createOrReplaceTempView("d4")

keep = df3.columns[0] + "," + df3.columns[1] + "," + df3.columns[2] + "," + df4.columns[2]

query = "SELECT {} FROM d3 d LEFT JOIN d4 e ON d.date = e.ix0 AND d.Action = e.ix1".format(keep)

df7 = sqlContext.sql(query)

In [ ]:
df7 = sqlContext.sql(query)

df7.show()


In [ ]:
from hdfs import Config

######## CONFIG #######################""
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/source/"



client = Config().get_client()
FilesList = client.list('/user/cluster/TestCorrel/source')
total = len(FilesList)

first = FilesList.pop(0)

#### INIT ##################################
dfW = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "PERMISSIVE")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv(SOURCE_DIR + first)
    
dfW = dfW.withColumnRenamed('_c0','date')
dfW = dfW.withColumnRenamed('_c1','Action')
i=0

for file_ in FilesList:
    i=i+1
    print("####################################")
    print("Work: " + str(i) + "/" + str(total))
    print("####################################")
    print(file_)

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)

    
    dftemp = dftemp.withColumnRenamed('_c0','ix0')
    dftemp = dftemp.withColumnRenamed('_c1','ix1')

    dfW.createOrReplaceTempView("fW")
    dftemp.createOrReplaceTempView("ftemp")

    keep = ','.join(map(str, dfW.columns))  + "," + dftemp.columns[2]

    query = "SELECT {} FROM fW d LEFT JOIN ftemp e ON d.date = e.ix0 AND d.Action = e.ix1".format(keep)

    
    dfW = sqlContext.sql(query)

    
##### STORAGE
dfW.write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/_BigMatrix__.csv")


In [ ]:
df7.coalesce(1)\
    .write\
    .format("csv")\
    .option("header", "true")\
    .save("./output2.csv")\
    

In [ ]:
dfW.write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/pBigM_.csv")

In [ ]:
dfW.coalesce(1)\
    .write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/_Big.csv")

In [ ]:
dfW.cache()

In [ ]:
from pyspark.mllib.stat import Statistics

L_df = dfW.columns
a = dfW.stat.corr(L_df[2],L_df[3])
print(a)

In [ ]:
dfW.repartition(64)

In [ ]:
from hdfs import Config

######## CONFIG #######################""
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/source/"



client = Config().get_client()
FilesList = client.list('/user/cluster/TestCorrel/source')
FilesList = FilesList[:100]
total = len(FilesList)

first = FilesList.pop(0)

#### INIT ##################################
dfW = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "PERMISSIVE")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv(SOURCE_DIR + first)
    
dfW = dfW.withColumnRenamed('_c0','date')
dfW = dfW.withColumnRenamed('_c1','Action')
i=0
for file_ in FilesList:
    i=i+1
    print("####################################")
    print("Work: " + str(i) + "/" + str(total))
    print("####################################")
    print(file_)

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)

    
    dftemp = dftemp.withColumnRenamed('_c0','ix0')
    dftemp = dftemp.withColumnRenamed('_c1','ix1')

    dfW.createOrReplaceTempView("fW")
    dftemp.createOrReplaceTempView("ftemp")

    keep = ','.join(map(str, dfW.columns))  + "," + dftemp.columns[2]

    query = "SELECT {} FROM fW d LEFT JOIN ftemp e ON d.date = e.ix0 AND d.Action = e.ix1".format(keep)

    
    dfW = sqlContext.sql(query)

    
'''
dfW.write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/_BigMatrix__.csv")
'''

In [ ]:
from pyspark.mllib.stat import Statistics

dfW.repartition(64)
L_df = dfW.columns
a = dfW.stat.corr(L_df[2],L_df[3])
print(a)

In [ ]:
dfSave = dfW

In [ ]:
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
######## CONFIG #######################""
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/source/"

num_cores = 16

t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/TestCorrel/source')
FilesList = FilesList[:50]
total = len(FilesList)

first = FilesList.pop(0)

#### INIT ##################################
dfW = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "PERMISSIVE")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv(SOURCE_DIR + first)
    
dfW = dfW.withColumnRenamed('_c0','date')
dfW = dfW.withColumnRenamed('_c1','Action')
i=0

dfW.repartition(7*num_cores)
for file_ in FilesList:
    i=i+1
    print("####################################")
    print("Work: " + str(i) + "/" + str(total))

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)

    
    dftemp = dftemp.withColumnRenamed('_c0','ix0')
    dftemp = dftemp.withColumnRenamed('_c1','ix1')

    dfW.createOrReplaceTempView("fW")
    dftemp.createOrReplaceTempView("ftemp")

    keep = ','.join(map(str, dfW.columns))  + "," + dftemp.columns[2]

    query = "SELECT {} FROM fW d LEFT JOIN ftemp e ON d.date = e.ix0 AND d.Action = e.ix1".format(keep)

    
    dfW = sqlContext.sql(query)
    dfW.repartition(7*num_cores)
    if (i % 10 == 0):
        dfW.persist(StorageLevel.DISK_ONLY)
        
    
'''
dfW.write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/TestCorrel/_BigMatrix__.csv")



L_df = dfW.columns
a = dfW.stat.corr(L_df[2],L_df[3])
print(a)

dfW.cache()
b = dfW.stat.corr(L_df[2],L_df[8])
print(b)
'''
t2 = time.time()
print("####################################")
print(t2-t1)


In [ ]:
t1 = time.time()
dfW.cache()
t2 = time.time()
print(t2-t1)

In [ ]:
import numpy as np

t1 = time.time()
L_df = dfW.columns
s = len(L_df)
t = s - 2

Mat_cor = np.ones((t,t))


for i in range(t):
    for j in range(i+1,t):
        print("i:"+str(i)+" j:"+str(j))
        Mat_cor[i,j] = dfW.stat.corr(L_df[i + 2], L_df[j+2])
        Mat_cor[j,i] = Mat_cor[i,j]
        print("correl: "+str(Mat_cor[i,j]))
        
t2 = time.time()
print(t2-t1)
Mat_cor

In [ ]:
import pandas as pd

In [ ]:

L_df = dfW.columns
colsRows = L_df[2:]

dfSave =dfW

pd_df = pd.DataFrame(data=Mat_cor, columns=colsRows, index=colsRows)

pd_df.to_csv("./testCor.csv")

In [ ]:
dfSave =dfW

In [ ]:
import time
dfSave.cache()

In [ ]:
dfSave.cache()
b = dfW.stat.corr(L_df[2],L_df[8])
print(b)

In [ ]:
b = dfW.stat.corr(L_df[3],L_df[82])
b

In [ ]:
dfW

In [ ]:
dfW.rdd

In [ ]:
dfW.coalesce(1).write\
    .format("csv")\
    .option("header", "true")\
    .save("hdfs://ad-cluster-master:54310/user/cluster/foo_export.csv")

In [ ]:
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
######## CONFIG #######################""
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"

num_cores = 16

t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')

first = FilesList.pop(0)

#### INIT ##################################
dfW = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "PERMISSIVE")\
    .option("inferSchema", "true")\
    .option("delimiter", ",")\
    .csv(SOURCE_DIR + first)
    
dfW.show()

In [ ]:
dfW.show()

In [ ]:
dfW.rdd.persist(StorageLevel.MEMORY_AND_DISK)
L = []
L.append(dfW.rdd)
dfW.rdd.repartition

In [ ]:
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
######## CONFIG #######################""
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"

num_cores = 16

t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
FilesList = FilesList[:10]
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("####################################")
    print("Work: " + str(i) + "/" + str(total))

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)
    
    rddT = dftemp.rdd
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    rddT = rddT.repartition(3*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
len(L)

In [ ]:
from pyspark.mllib.stat import Statistics
a = L[1].collect()
b = L[2].collect()



In [ ]:
L[1]

In [ ]:
x = sc.parallelize(L[1])
y = sc.parallelize(L[2])

In [ ]:
c = L[1].map(list)
e = L[2].map(list)

In [ ]:
d = c.collect()
f = e.collect()

In [ ]:
g = L[1].map(tuple)
g

In [ ]:
h = g.collect()

In [ ]:
seriesX = sc.parallelize([1.0, 2.0, 3.0, 3.0, 5.0])  # a series
# seriesY must have the same number of partitions and cardinality as seriesX
seriesY = sc.parallelize([11.0, 22.0, 33.0, 33.0, 555.0])

In [ ]:
seriesX

In [ ]:
from pyspark.mllib.stat import Statistics
a = L[1]
b = L[2]
a

In [ ]:
a.count()

In [ ]:
Statistics.corr(a,a)

In [ ]:
y = sc.parallelize(a.map(list).collect())

In [ ]:
y.sum()

In [ ]:
rT = dftemp.rdd.map(lambda x: x[0])
rT.persist(StorageLevel.MEMORY_AND_DISK)
Statistics.corr(rT,rT)

In [ ]:
L

In [ ]:
a.collect()

In [ ]:
a=L[2]
b=L[3]

b

In [ ]:
Statistics.corr(x=L[5],y=L[3])

In [ ]:
Statistics.corr(x=L[2],y=L[3])

In [ ]:
Statistics.corr(x=L[4],y=L[8])

In [ ]:
import numpy as np

t1 = time.time()
L_rdd = L
s = len(L_rdd)
t = s

Mat_cor = np.ones((t,t))


for i in range(t):
    for j in range(i+1,t):
        print("i:"+str(i)+" j:"+str(j))
        Mat_cor[i,j] = Statistics.corr(L_rdd[i], L_rdd[j])
        Mat_cor[j,i] = Mat_cor[i,j]
        print("correl: "+str(Mat_cor[i,j]))
        
t2 = time.time()
print(t2-t1)
Mat_cor

In [ ]:
a = L[1]
b = L[2]

In [ ]:
a = L[1]
a

In [ ]:
a.collect()

In [ ]:
b = a.partitionBy(100)
b

In [ ]:
c = b.map(tuple)

In [ ]:
c.collect()

In [ ]:
L[1].collect()

In [ ]:
def count_in_a_partition(iterator):
  yield sum(1 for _ in iterator)

print(L[1].mapPartitions(count_in_a_partition).collect())
print(L[2].mapPartitions(count_in_a_partition).collect())
print(L[3].mapPartitions(count_in_a_partition).collect())
print(L[4].mapPartitions(count_in_a_partition).collect())
print(L[5].mapPartitions(count_in_a_partition).collect())
print(L[6].mapPartitions(count_in_a_partition).collect())
print(L[7].mapPartitions(count_in_a_partition).collect())
print(L[8].mapPartitions(count_in_a_partition).collect())
print(L[9].mapPartitions(count_in_a_partition).collect())
print(L[0].mapPartitions(count_in_a_partition).collect())

In [ ]:
print(L[0].mapPartitions(count_in_a_partition).collect())
a = L[0].repartition(4)
print(a.mapPartitions(count_in_a_partition).collect())
b = a.repartition(4)
print(a.mapPartitions(count_in_a_partition).collect())
c = b.repartition(4)
print(a.mapPartitions(count_in_a_partition).collect())
d = c.repartition(4)
print(a.mapPartitions(count_in_a_partition).collect())
e = d.repartition(4)
print(a.mapPartitions(count_in_a_partition).collect())

In [ ]:
BigL = L[0].map(lambda x: x[0])
print(BigL.mapPartitions(count_in_a_partition).collect())
f = BigL.repartition(4)
print(f.mapPartitions(count_in_a_partition).collect())

BigL2 = L[1].map(lambda x: x[0])
print(BigL2.mapPartitions(count_in_a_partition).collect())
g = BigL2.repartition(4)
print(g.mapPartitions(count_in_a_partition).collect())

In [ ]:
Statistics.corr(f,g)

In [ ]:
BigL = L[0].map(lambda x: x[0]).repartition(4)
print(BigL.mapPartitions(count_in_a_partition).collect())
f = BigL.repartition(4)
print(f.mapPartitions(count_in_a_partition).collect())

BigL2 = L[1].map(lambda x: x[0]).repartition(4)
print(BigL2.mapPartitions(count_in_a_partition).collect())
g  = BigL2.repartition(4)
print(g.mapPartitions(count_in_a_partition).collect())

In [ ]:
Statistics.corr(f,g)

In [ ]:
v = Vectors.dense(a)

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np

Z = []
for i in range(len(L)):
    s = np.array(L[i].map(lambda x: x[0]).collect())
    Z.append(s)

In [ ]:
TestRDD = sc.parallelize(Z)

In [ ]:
Z

In [ ]:
def count_in_a_partition(iterator):
  yield sum(1 for _ in iterator)

TestRDD
print(TestRDD.mapPartitions(count_in_a_partition).collect())
    
pdDF = pd.DataFrame(Z)

In [ ]:
pdDF.to_csv("./fooo.csv")

In [ ]:
Z[1]

In [ ]:
s = np.array(L[2].map(lambda x: x[0]).collect())
s

In [ ]:
np.array([1.0, 10.0, 100.0])

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#######################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:10] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)
    
    rddT = dftemp.rdd
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

t1 = time.time()
print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

Z = []
for i in range(len(L)):
    s = np.array(L[i].map(lambda x: x[0]).collect())
    Z.append(s)
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

t1 = time.time()
print("#############################")
print("COLLECT TASK FINISHED")
print("\n")

M = np.matrix(Z)
N = M.transpose()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

t1 = time.time()
print("#############################")
print("PIPE NUMPY TASK FINISHED")
print("\n")

NewRDD = sc.parallelize(np.asarray(N))
A = Statistics.corr(NewRDD)
A

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("JOB FINISHED")
print("\n")

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#######################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
FilesList = FilesList[:10] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)
    
    rddT = dftemp.rdd
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

L


In [ ]:
t1 = time.time()
print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

Z = np.empty(len(L),dtype=object)

for i in range(len(L)):
    print(i)
    a = L[i].map(lambda x: x[0]).collect()
    print(type(a))
    break
    Z[i] = np.array(a)
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("#############################")
print("COLLECT TASK FINISHED")
print("\n")

In [ ]:
t1 = time.time()
print(len(Z))
Z = Z.transpose()
print(len(Z))

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("#############################")
print("PIPE NUMPY TASK FINISHED")
print("\n")


In [ ]:
np.matrix(Z)
np.matrix(Z)

In [ ]:
t1 = time.time()

NewRDD = sc.parallelize(Z,4*20)
#A = Statistics.corr(NewRDD)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("JOB FINISHED")
print("\n")

In [ ]:
t1 = time.time()

A = Statistics.corr(NewRDD)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("JOB FINISHED")
print("\n")

In [ ]:
A

In [ ]:
t1 = time.time()

M = np.matrix(Z[:50])
N = M.transpose()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("#############################")
print("PIPE NUMPY TASK FINISHED")
print("\n")

t1 = time.time()

NewRDD = sc.parallelize(np.asarray(N),4*num_cores)
A = Statistics.corr(NewRDD)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("JOB FINISHED")
print("\n")

In [ ]:
np.savetxt("./supfoot.csv",A,delimiter=",")

In [ ]:
np.savetxt("./supsupfoot.csv",N,delimiter=",")

In [ ]:
ne

In [ ]:
t1 = time.time()
print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

Z = []

for i in range(len(L)):
    Z.append(L[i].map(lambda x: x[0]))


t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))
print("#############################")
print("COLLECT TASK FINISHED")
print("\n")
Z


In [ ]:
A = L[0].map(lambda x: np.asarray(x))
A.collect()

In [ ]:
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


In [ ]:
R4 = rddTranspose(L[0])

In [ ]:
K = []
for i in range(len(L)):
    print(i)
    K.append(rddTranspose(L[i]))



In [ ]:
C = O.map(lambda x: np.asarray(x))
E = C.collect()
E

In [ ]:
P = sc.union(L[:2])
P.persist()

In [ ]:
D = P.map(lambda x: np.asarray(x))
F = D.collect()
F

In [ ]:
O = sc.union(K)
O.persist()

In [ ]:
Q = rddTranspose(O)


In [ ]:
R = Q.map(lambda x: np.asarray(x))
G = R.collect()
G

In [ ]:
Out = Statistics.corr(Q)

In [ ]:
tex = np.asarray(G)

np.savetxt("./Outfoo.csv", tex, delimiter=",")

np.savetxt("./OutCorr.csv", Out, delimiter=",")

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:10] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    dftemp = spark.read.format("csv")\
            .option("header", "true")\
            .option("mode", "PERMISSIVE")\
            .option("inferSchema", "true")\
            .option("delimiter", ",")\
            .csv(SOURCE_DIR + file_)
    
    rddT = dftemp.rdd.repartition(4*num_cores)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


t1 = time.time()

K = []
total = len(L)
for i in range(total):
    print("Transpose: " + str(i) + "/" + str(total))
    K.append(rddTranspose(L[i]))
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FIRST TRANSPOSE TASK FINISHED")
print("\n")

####################################################################
t1 = time.time()


O = sc.union(K)
print("UNION OVER")
Q = rddTranspose(O)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("UNION AND SECOND TRANSPOSE TASK FINISHED")
print("\n")

'''########################## DEBUG #########################################
t1 = time.time()
R = Q.map(lambda x: np.asarray(x))
R.persist(StorageLevel.MEMORY_AND_DISK)
G = R.collect()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("SECOND TRANSPOSE TASK FINISHED")
print("\n")
'''

####################################################################
t1 = time.time()
Out = Statistics.corr(Q)
t2 = time.time()

print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("CORRELATION COMPUTATION FINISHED")
print("\n")

In [ ]:
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


t1 = time.time()

K = []
total = len(L)
for i in range(total):
    print("Transpose: " + str(i+1) + "/" + str(total))
    K.append(rddTranspose(L[i]))
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FIRST TRANSPOSE TASK FINISHED")
print("\n")

####################################################################
t1 = time.time()


O = sc.union(K)
print("UNION OVER")
Q = rddTranspose(O)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("UNION AND SECOND TRANSPOSE TASK FINISHED")
print("\n")

'''########################## DEBUG #########################################
t1 = time.time()
R = Q.map(lambda x: np.asarray(x))
R.persist(StorageLevel.MEMORY_AND_DISK)
G = R.collect()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("SECOND TRANSPOSE TASK FINISHED")
print("\n")
'''

####################################################################
t1 = time.time()
Out = Statistics.corr(Q)
t2 = time.time()

print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("CORRELATION COMPUTATION FINISHED")
print("\n")

In [ ]:
from itertools import islice

rddFile = sc.textFile(SOURCE_DIR + FilesList[0],4*num_cores)

PL = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)

In [ ]:
PL.getNumPartitions()

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:10] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    
    rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")

####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


t1 = time.time()

K = []
total = len(L)
for i in range(total):
    print("Transpose: " + str(i+1) + "/" + str(total))
    K.append(rddTranspose(L[i]))
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FIRST TRANSPOSE TASK FINISHED")
print("\n")

####################################################################
t1 = time.time()


O = sc.union(K)
print("UNION OVER")
Q = rddTranspose(O)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("UNION AND SECOND TRANSPOSE TASK FINISHED")
print("\n")

'''########################## DEBUG #########################################
t1 = time.time()
R = Q.map(lambda x: np.asarray(x))
R.persist(StorageLevel.MEMORY_AND_DISK)
G = R.collect()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("SECOND TRANSPOSE TASK FINISHED")
print("\n")
'''

####################################################################
t1 = time.time()
Out = Statistics.corr(Q)
t2 = time.time()

print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("CORRELATION COMPUTATION FINISHED")
print("\n")

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/workfileBis/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/workfileBis')
#FilesList = FilesList[:10] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    rddT = rddFile
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
L[4].map(lambda x: np.asarray(x)).collect()

In [ ]:
O = sc.union(L)
print("UNION OVER")
O

In [ ]:
####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4

In [ ]:
t1 = time.time()
Q = rddTranspose(O)
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

In [ ]:
####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


t1 = time.time()

K = []
total = len(L)
for i in range(total):
    print("Transpose: " + str(i+1) + "/" + str(total))
    K.append(rddTranspose(L[i]))
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FIRST TRANSPOSE TASK FINISHED")
print("\n")



In [ ]:
####################################################################
t1 = time.time()


O = sc.union(K)
print("UNION OVER")
Q = rddTranspose(O)

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("UNION AND SECOND TRANSPOSE TASK FINISHED")
print("\n")

'''########################## DEBUG #########################################
t1 = time.time()
R = Q.map(lambda x: np.asarray(x))
R.persist(StorageLevel.MEMORY_AND_DISK)
G = R.collect()

t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("SECOND TRANSPOSE TASK FINISHED")
print("\n")
'''

####################################################################
t1 = time.time()
Out = Statistics.corr(Q)
t2 = time.time()

print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("CORRELATION COMPUTATION FINISHED")
print("\n")

In [ ]:
O

In [ ]:
O.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/workfileBis/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/workfileBis')
FilesList = FilesList[:3] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_ ,4*num_cores)
    rddT = rddFile.map(lambda x: x.split(",")[1:])
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
import numpy as np
from pyspark.mllib.linalg import Vectors

rddFile = sc.textFile(SOURCE_DIR + FilesList[0],4*num_cores)
rddT = rddFile.map(lambda x: x.split(",")[1:])



In [ ]:
####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4

rddTtr = rddTranspose(rddT)

In [ ]:
DF = rddT.toDF()
DF.show()

In [ ]:
DF2 = rddTtr.toDF()
DF2.show()

In [ ]:
import numpy as np
from pyspark.mllib.linalg import Vectors


sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])

sv1

In [ ]:
U = sc.union(L)
U

DfU = U.toDF()

In [ ]:
DfU.show()

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/sample/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/sample')
FilesList = FilesList[:3] #Debug
total = len(FilesList)

L=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_ ,4*num_cores)
    #rddT = rddFile.map(lambda x: x.split(",",1)[1])
    rddT = rddFile.flatMap(lambda x: (x.split(",")[0],\
                                    map(float,x.split(",")[1:])\
                                  ))
    #rddT = rddFile.map(lambda x: (x.split(",")[0],x.split(",")[1:]))
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    L.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
for i in range(len(L)):
    print(L[i].collect())

In [ ]:
for i in range(len(L)):
    print(L[i].collect())

In [ ]:
for i in range(len(L)):
    L[i].map(lambda x: np.asarray(x))
    print(L[i].collect())

In [ ]:
####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4


In [ ]:
U = sc.union(L)
U.collect()

#rddTranspose(U).collect()
#DfU = U.toDF()

In [ ]:

DfU.show()

In [ ]:
Statistics.corr(U)

In [ ]:
U.map(lambda x: np.asarray(x)).collect()

In [ ]:
V = U.flatMap(lambda l: l)

In [ ]:
U.collect()

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:3] #Debug
total = len(FilesList)

I=[]
i=0
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    rddT = rddFile.zipWithIndex().map(lambda (x,y): (y,x))
    #rddT = rddFile.map(lambda x: x.split(",",1)[1])
    #rddT = rddFile.flatMap(lambda x: (x.split(",")[0],\
    #                                map(float,x.split(",")[1:])\
    #                              ))
    #rddT = rddFile.map(lambda x: (x.split(",")[0],x.split(",")[1:]))
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    I.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
####################################################################
def rddTranspose(rdd):
    rddT1 = rdd.zipWithIndex().flatMap(lambda (x,i): [(i,j,e) for (j,e) in enumerate(x)])
    rddT2 = rddT1.map(lambda (i,j,e): (j, (i,e))).groupByKey().sortByKey()
    rddT3 = rddT2.map(lambda (i, x): sorted(list(x), cmp=lambda (i1,e1),(i2,e2) : cmp(i1, i2)))
    rddT4 = rddT3.map(lambda x: map(lambda (i, y): y , x))
    return rddT4
print(I[0].collect())
print(I[1].collect())
a = I[0]
b = a.join(I[1])
print(I[2].collect())
c = b.join(I[2])

In [ ]:
copy
enumerate(copy)

In [ ]:
c.groupByKey().collect()

In [ ]:
uni_ = sc.union(I)

fin = uni_.groupByKey().mapValues(tuple).map(lambda x: x[1])


header = list(fin.take(1)[0])

#tup1 = index created
#tup2 = data
#map data to float 
end = fin.zipWithIndex().filter(lambda tup: tup[1] > 0).map(lambda tup: map(float,tup[0]))
Mat = end.toDF(header)





In [ ]:
uni_ = sc.union(I)

In [ ]:
fin = uni_.groupByKey().mapValues(tuple).filter(lambda tup: tup[1] > 0).map(lambda x: x[1]).map(lambda tup: map(float,tup[0]))

In [ ]:
header = list(fin.take(1)[0])

In [ ]:
list(fin.take(1)[0])

In [ ]:
end = fin.zipWithIndex()

In [ ]:
Mat = end.toDF(header)

In [ ]:
header
Lo = []
for i in range(len(FilesList)):
    temp = FilesList[i][:-4]
    f_c_ = ''.join(e for e in temp if e.isalnum())
    Lo.append(f_c_)

In [ ]:
header

In [ ]:
Lo

In [ ]:
I[0].collect()

In [ ]:
Mat.stat.corr(str(header[0]),str(header[1]))

In [ ]:
for t in enumerate(I):
    print t

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:3] #Debug
total = len(FilesList)

I=[]
i=0
header = []
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    header.append(rddFile.take(1)[0])
    
    rddT = rddFile.zipWithIndex().map(lambda (x,y): (y,x))
    
    #rddT = rddFile.map(lambda x: x.split(",",1)[1])
    #rddT = rddFile.flatMap(lambda x: (x.split(",")[0],\
    #                                map(float,x.split(",")[1:])\
    #                              ))
    #rddT = rddFile.map(lambda x: (x.split(",")[0],x.split(",")[1:]))
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    I.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
uni_ = sc.union(I)

In [ ]:
#tup0 = index created
#tup1 = data
#map data to float 
fin = uni_.groupByKey().mapValues(tuple)\
                        .filter(lambda tup: tup[0] > 0)\
                        .map(lambda tup: map(float,tup[1]))

In [ ]:
s = fin.persist(StorageLevel.MEMORY_AND_DISK)

In [ ]:
Mat2 = s.toDF(header)


In [ ]:
fn

In [ ]:
p.take(1)

In [ ]:
import numpy as np
Mat2.stat.corr(str(header[0]),str(header[1]))

In [ ]:
Array = np.array(Statistics.corr(s))

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/source_wfile/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/source_wfile')
#FilesList = FilesList[:3] #Debug
total = len(FilesList)

I=[]
i=0
header = []
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    header.append(rddFile.take(1)[0])
    
    rddT = rddFile.zipWithIndex().filter(lambda col: col[1] > 0).map(lambda x: (map(float,x)[0],x[1]))

    df = rddT.toDF([header[i-1],"uuid"])
    
    #rddT = rddFile.map(lambda x: x.split(",",1)[1])
    #rddT = rddFile.flatMap(lambda x: (x.split(",")[0],\
    #                                map(float,x.split(",")[1:])\
    #                              ))
    #rddT = rddFile.map(lambda x: (x.split(",")[0],x.split(",")[1:]))
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    I.append(df)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
I[325].show()

In [ ]:
op = I[0]

for i in range(len(FilesList) - 1):
    print("Task #" + str(i) + "/" + str(len(FilesList)))
    op = op.join(I[i+1], "uuid")



In [ ]:
#op.collect()
op.stat.corr(str(header[82]), str(header[125]))

In [ ]:
I[1].uuid

In [ ]:
from pyspark.mllib.linalg import Vector, Vectors
import numpy as np
from pyspark.mllib.stat import Statistics
import pandas as pd
from hdfs import Config
from pyspark.mllib.stat import Statistics
import time
from itertools import islice


######## CONFIG #######################
SOURCE_DIR = "hdfs://ad-cluster-master:54310/user/cluster/sample/col/"
num_cores = 16


#########################################################################
t1 = time.time()
client = Config().get_client()
FilesList = client.list('/user/cluster/sample/col')
#FilesList = FilesList[:3] #Debug
total = len(FilesList)

I=[]
i=0
header = []
for file_ in FilesList:
    i=i+1
    print("#########")
    print("Work: " + str(i) + "/" + str(total))

    rddFile = sc.textFile(SOURCE_DIR + file_,4*num_cores)
    header.append(rddFile.take(1)[0])
    
    rddT = rddFile.zipWithIndex().map(lambda (x,y): (y,x))
    
    #rddT = rddFile.map(lambda x: x.split(",",1)[1])
    #rddT = rddFile.flatMap(lambda x: (x.split(",")[0],\
    #                                map(float,x.split(",")[1:])\
    #                              ))
    #rddT = rddFile.map(lambda x: (x.split(",")[0],x.split(",")[1:]))
    #rddT = rddFile.mapPartitionsWithIndex(lambda i, iter: islice(iter, 1, None) if i == 0 else iter)
    #print(rddT)
    #rddT = dftemp.rdd.map(lambda x: x[0])
    #rddT = rddT.repartition(1)
    #rddT = rddT.repartition(4*num_cores)
    #rddT.persist(StorageLevel.MEMORY_AND_DISK)
    I.append(rddT)
    
    
    '''dftemp.repartition(3*num_cores)
    dftemp.persist(StorageLevel.MEMORY_AND_DISK)
    P.append(dftemp)'''
    
t2 = time.time()
print("####################################")
print("TIME TAKEN FOR TASK:" + str(t2-t1))

print("#############################")
print("FILE LOADING TASK FINISHED")
print("\n")



In [ ]:
uni_ = sc.union(I)

In [ ]:
#tup0 = index created
#tup1 = data
#map data to float 
fin = uni_.groupByKey().mapValues(tuple)\
                        .filter(lambda tup: tup[0] > 0)\
                        .map(lambda tup: map(float,tup[1]))

In [ ]:
fin.collect()

In [ ]:
spark